In [ ]:
# ===================================================================
# CELL 1: SETUP SECRETS AND EXECUTE PIPELINE
# ===================================================================
import os
import base64
from kaggle_secrets import UserSecretsClient

print("--- 1. Setting up secrets and environment ---")
user_secrets = UserSecretsClient()
os.environ['GIT_DEPLOY_KEY_B64'] = user_secrets.get_secret("GIT_DEPLOY_KEY_B64")
os.environ['GDRIVE_CREDENTIALS_DATA_B64'] = user_secrets.get_secret("GDRIVE_CREDENTIALS_DATA_B64")

# --- 2. Set up SSH for Git ---
ssh_dir = "/root/.ssh"
os.makedirs(ssh_dir, exist_ok=True)
key_path = os.path.join(ssh_dir, "id_ed25519")
with open(key_path, "w") as f:
    f.write(base64.b64decode(os.environ['GIT_DEPLOY_KEY_B64']).decode() + '\n')
os.chmod(key_path, 0o600)
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# --- 3. Clone Git Repository using SSH ---
print("\n--- 2. Cloning repository via SSH ---")
GIT_REPO_URL = "git@github.com:labyedh/Alzheimer_classification_mlops.git"
PROJECT_DIR = "Alzheimer_classification_mlops"
!rm -rf {PROJECT_DIR}
!git clone {GIT_REPO_URL}
%cd {PROJECT_DIR}

# --- 4. Execute the pipeline script ---
print(f"\n--- 3. Executing pipeline script in {os.getcwd()} ---")
!chmod +x run_pipeline.sh
!./run_pipeline.sh